In [1]:
from scripts.data_util.sqlConnector import online_sql_connector , update_sql_since
connector = online_sql_connector()
update_sql_since('SellSideFactors' , connector = connector , trace = 0)
connector.close_all()

Sat Jan 27 23:37:28 2024 : haitong/hf_factors since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:37:37 2024 : haitong/dl_factors since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:38:24 2024 : dongfang/hfq_chars since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:38:42 2024 : dongfang/l2_chars since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:40:27 2024 : dongfang/ms_chars since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:43:03 2024 : dongfang/order_flow since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:43:12 2024 : dongfang/gp since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:43:15 2024 : dongfang/tra since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:43:20 2024 : dongfang/scores_v0 since 20240106 to 20240127, total 1 periods(Q)
Sat Jan 27 23:43:22 2024 : kaiyuan/positive since 20240101 to 20240127, total 1 periods(Q)
Sat Jan 27 23:43:27 2024 : kaiyuan/negative since 20231230 to 20240127, total 1 period

In [3]:
#from scripts.data_util.DataTank import *
#from scripts.data_util.DataUpdater import *
#repack_DB_bygroup('data/DB_data/DB_3rdPartySQL.h5' , 'SellSideFactors' , True)